<a href="https://colab.research.google.com/github/asmaharrabi/RAG-Project2025/blob/main/CRAG%2BSTARK%2BORCHESTRATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CELLULE 1 : Setup Initial
# ========================================
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/CRAG_PROJECT
%cd /content/CRAG_PROJECT

# CRAG → GitHub
![ ! -d "CRAG" ] && git clone https://github.com/HuskyInSalt/CRAG.git

# STARK → TON PC (Drive)
!cp -r "/content/drive/MyDrive/STARK_FROM_PC/stark" ./stark 2>/dev/null || echo "STARK pas trouvé"

# Sauvegarde auto
!rsync -a CRAG stark /content/drive/MyDrive/CRAG_FULL_BACKUP/ --progress

print("✅ CRAG + STARK PRÊTS")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/CRAG_PROJECT
sending incremental file list
CRAG/
CRAG/.git/
CRAG/.git/branches/
CRAG/.git/hooks/
CRAG/.git/info/
CRAG/.git/logs/
CRAG/.git/logs/refs/
CRAG/.git/logs/refs/heads/
CRAG/.git/logs/refs/remotes/
CRAG/.git/logs/refs/remotes/origin/
CRAG/.git/objects/
CRAG/.git/objects/info/
CRAG/.git/objects/pack/
CRAG/.git/refs/
CRAG/.git/refs/heads/
CRAG/.git/refs/remotes/
CRAG/.git/refs/remotes/origin/
CRAG/.git/refs/tags/
CRAG/data/
CRAG/data/arc_challenge/
CRAG/data/arc_challenge/ref/
CRAG/data/bio/
CRAG/data/bio/ref/
CRAG/data/popqa/
CRAG/data/popqa/ref/
CRAG/data/pubqa/
CRAG/data/pubqa/ref/
CRAG/img/
CRAG/scripts/
stark/stark/.gitignore
            251 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=111/231)
stark/stark/LICENSE
          1,068 100%    1.02MB/s    0:00:00 (xfr#2, to-chk=110/231)
stark/stark/README.md
         10,429 100%    9.95MB/s    0:0

In [ ]:
# CELLULE 2A : NETTOYAGE COMPLET
# ============================================================================
print("🧹 NETTOYAGE COMPLET DE L'ENVIRONNEMENT...")

!pip uninstall -y torch torchvision torchaudio torch-geometric torch-scatter torch-sparse torch-cluster transformers sentence-transformers accelerate -q

print("✅ Nettoyage terminé")
print("\n⚠️⚠️⚠️ IMPORTANT ⚠️⚠️⚠️")
print("👉 Menu Runtime → Restart runtime")
print("👉 PUIS exécute la CELLULE 2B")

🧹 NETTOYAGE COMPLET DE L'ENVIRONNEMENT...
✅ Nettoyage terminé

⚠️⚠️⚠️ IMPORTANT ⚠️⚠️⚠️
👉 Menu Runtime → Restart runtime
👉 PUIS exécute la CELLULE 2B


In [ ]:
# CELLULE 2B : INSTALLATION COMPLÈTE (APRÈS RESTART)
# ============================================================================
print("🔥 INSTALLATION PYTORCH GPU + DÉPENDANCES...")

# 1. PyTorch GPU
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu118

# 2. Transformers (VERSION STABLE)
!pip install transformers==4.36.2 accelerate

# 3. Sentence Transformers
!pip install sentence-transformers

# 4. Autres dépendances
!pip install qdrant-client googlesearch-python requests beautifulsoup4 pydantic networkx

print("\n✅ VÉRIFICATION GPU ET PACKAGES...")

import torch
print(f"\n{'='*60}")
print(f"🔥 GPU ACTIF : {torch.cuda.is_available()}")
print(f"📦 TORCH VERSION : {torch.__version__}")
if torch.cuda.is_available():
    print(f"🎮 GPU DÉTECTÉ : {torch.cuda.get_device_name(0)}")
    print(f"💾 MÉMOIRE GPU : {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("\n❌ GPU NON DÉTECTÉ !")
    print("👉 Runtime → Change runtime type → T4 GPU")

import transformers
print(f"📦 Transformers : {transformers.__version__}")

import accelerate
print(f"📦 Accelerate : {accelerate.__version__}")

print(f"{'='*60}\n")

if not torch.cuda.is_available():
    raise RuntimeError("⚠️ GPU non actif - Change le runtime!")

print("✅ TOUTES LES DÉPENDANCES INSTALLÉES CORRECTEMENT")

🔥 INSTALLATION PYTORCH GPU + DÉPENDANCES...
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp312-cp312-linux_x86_64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp312-cp312-linux_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.6.0%2Bcu118-cp312-cp312-linux_x86_64.whl.metadata (6.6 kB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp312-cp312-linux_x86_64.whl (848.7 MB)
Using cached https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp312-cp312-linux_x86_64.whl (6.5 MB)
Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.6.0%2Bcu118-cp312-cp312-linux_x86_64.whl (3.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft

In [ ]:
# CELLULE 2C : SETUP BGE-M3 + QDRANT
# ========================================
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
import uuid
import torch

print("🔍 Vérification GPU...")
print(f"GPU disponible : {torch.cuda.is_available()}")

# Qdrant en RAM
client = QdrantClient(":memory:")
client.create_collection(
    collection_name="docs",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
)
print("✅ Qdrant initialisé")

# BGE-M3 sur GPU
print("\n📥 Chargement BGE-M3 sur GPU...")
model = SentenceTransformer('BAAI/bge-m3', device='cuda')
print(f"✅ BGE-M3 sur : {model.device}")

# Test
texts = ["Nanofluids boost heat transfer by 40% with copper oxide."]
points = [
    PointStruct(
        id=str(uuid.uuid4()),
        vector=model.encode(t, normalize_embeddings=True).tolist(),
        payload={"content": t}
    )
    for t in texts
]
client.upsert("docs", points)

print("\n✅ SYSTÈME PRÊT : GPU + BGE-M3 + QDRANT")


🔍 Vérification GPU...
GPU disponible : True
✅ Qdrant initialisé

📥 Chargement BGE-M3 sur GPU...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

✅ BGE-M3 sur : cuda:0

✅ SYSTÈME PRÊT : GPU + BGE-M3 + QDRANT


In [ ]:
# CELLULE 3 : Indexation CRAG
# ========================================
import glob

# Recréer collection
try:
    client.delete_collection("docs")
except:
    pass

client.create_collection(
    collection_name="docs",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
)

# Charger passages CRAG
texts = []
for path in glob.glob("CRAG/data/**/ref/*", recursive=True):
    try:
        with open(path, encoding="utf-8", errors='ignore') as f:
            for line in f:
                line = line.strip()
                if line and len(line) > 15:
                    texts.append(line)
    except:
        continue

print(f"📚 {len(texts)} passages CRAG trouvés")

# Indexer par batch
batch_size = 100
max_docs = 500

for i in range(0, min(len(texts), max_docs), batch_size):
    batch = texts[i:i+batch_size]
    points = [
        PointStruct(
            id=str(uuid.uuid4()),
            vector=model.encode(t, normalize_embeddings=True).tolist(),
            payload={"content": t, "src": "CRAG"}
        )
        for t in batch
    ]
    client.upsert("docs", points)
    print(f"✅ Indexé {i+len(batch)}/{min(len(texts), max_docs)}")

print(f"\n✅ {min(len(texts), max_docs)} passages indexés")



📚 12253 passages CRAG trouvés
✅ Indexé 100/500
✅ Indexé 200/500
✅ Indexé 300/500
✅ Indexé 400/500
✅ Indexé 500/500

✅ 500 passages indexés


In [ ]:
# CELLULE 4 : Installation STARK (GPU SAFE)
# ========================================
print("🧹 Installation PyTorch Geometric...")

!pip uninstall -y torch-scatter torch-sparse torch-cluster torch-geometric -q
!pip install torch-geometric pyg_lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.6.0+cu118.html

print("\n📦 Installation STARK...")
!pip install -q --no-deps PyTDC==0.3.9
!pip install -q pandas numpy tqdm networkx huggingface_hub

import torch
print(f"\n✅ GPU toujours actif : {torch.cuda.is_available()}")
print("✅ Installation STARK terminée")


🧹 Installation PyTorch Geometric...
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00

📦 Installation STARK...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

✅ GPU toujours actif : True
✅ Installation STARK terminée


In [ ]:
# CELLULE 5 : Graphe STARK
# ========================================
import networkx as nx

# Graphe de connaissances thermique
G = nx.DiGraph()

entities_thermal = {
    "Nanofluid": ["Heat transfer", "Thermal conductivity"],
    "Copper oxide": ["Nanofluid", "Nanoparticles"],
    "Nanoparticles": ["Thermal conductivity", "Heat transfer"],
    "Heat transfer": ["Thermal conductivity"],
    "Thermal conductivity": ["Efficiency"]
}

for src, targets in entities_thermal.items():
    for tgt in targets:
        G.add_edge(src, tgt, relation="enhances")

print(f"✅ Graphe STARK : {G.number_of_nodes()} nœuds, {G.number_of_edges()} relations")
print(f"Entités : {list(G.nodes())}")



✅ Graphe STARK : 6 nœuds, 8 relations
Entités : ['Nanofluid', 'Heat transfer', 'Thermal conductivity', 'Copper oxide', 'Nanoparticles', 'Efficiency']


In [ ]:
# CELLULE 6 : LLM Gratuit (Groq API)
# ========================================
print("🤖 Configuration LLM gratuit...")

# Groq API (GRATUIT - 30 req/min)
import os
os.environ["GROQ_API_KEY"] = "YOUR_GROQ_API_KEY"  # Obtiens-le sur https://console.groq.com

# Alternative : Utiliser transformers local (plus lent mais 100% gratuit)
from transformers import pipeline

print("📥 Chargement Flan-T5-Large (gratuit, local)...")
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    device=0,  # GPU
    max_length=512
)
print("✅ Générateur prêt")

# Fonction wrapper
def generate_with_llm(prompt, max_tokens=150):
    """Génération avec Flan-T5"""
    result = generator(prompt, max_new_tokens=max_tokens, do_sample=False)
    return result[0]['generated_text'].strip()

# Test
test_response = generate_with_llm("What are nanofluids? Answer briefly:")
print(f"\n🧪 Test LLM: {test_response}")


🤖 Configuration LLM gratuit...
📥 Chargement Flan-T5-Large (gratuit, local)...


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


✅ Générateur prêt

🧪 Test LLM: nanofluids nanofluids may refer to:


In [ ]:
# CELLULE 7 : ORCHESTRATEUR INTELLIGENT
# ========================================
import re
import numpy as np
from typing import List, Tuple, Dict
from pydantic import BaseModel, field_validator
from googlesearch import search
import requests
from bs4 import BeautifulSoup

# === CLASSES ===
class EntiteExtraite(BaseModel):
    name: str
    type: str

class Action(BaseModel):
    domaine: str
    entites: List[EntiteExtraite]
    confidence: float
    selection_module: str

    @field_validator('selection_module')
    @classmethod
    def check(cls, v):
        return v if v in ['text', 'graph', 'web'] else 'text'

# === MODULES DE RÉCUPÉRATION ===

def analyze_question(q: str) -> Action:
    """Analyse intelligente de la question"""
    words = q.lower().split()

    # Détection domaine
    domaine = "General"
    confidence = 0.5

    if any(w in q.lower() for w in ["nano", "fluid", "heat", "thermal", "conductivity"]):
        domaine = "Thermal"
        confidence = 0.8
    elif any(w in q.lower() for w in ["battery", "energy", "electric", "power"]):
        domaine = "Energy"
        confidence = 0.8

    # Entités
    entites = [EntiteExtraite(name=w.capitalize(), type="ENTITY") for w in words if len(w) > 4][:3]

    # Sélection module initial
    if any(k in q.lower() for k in ["relation", "connect", "between", "compare"]):
        sel = "graph"
    elif any(k in q.lower() for k in ["latest", "recent", "news", "2024", "2025"]):
        sel = "web"
    else:
        sel = "text"

    return Action(domaine=domaine, entites=entites, confidence=confidence, selection_module=sel)


def retrieve_from_text(q: str, top_k: int = 6) -> List[Tuple[str, float]]:
    """Récupération vectorielle Qdrant"""
    vec = model.encode(q, normalize_embeddings=True).tolist()
    hits = client.search(
        collection_name="docs",
        query_vector=vec,
        limit=top_k,
        with_payload=True,
        search_params={"exact": False, "hnsw_ef": 128}
    )
    return [(h.payload["content"], h.score) for h in hits]


def retrieve_from_graph(q: str, top_k: int = 3) -> List[Tuple[str, float]]:
    """Récupération depuis le graphe STARK"""
    results = []
    words = q.lower().split()

    # Trouver entités mentionnées
    for node in G.nodes():
        if node.lower() in q.lower():
            # Récupérer voisins
            neighbors = list(G.neighbors(node))
            for neighbor in neighbors[:top_k]:
                edge_data = G.get_edge_data(node, neighbor)
                relation = edge_data.get('relation', 'related')
                doc = f"{node} {relation} {neighbor} in thermal systems."
                results.append((doc, 0.75))

    return results[:top_k]


def retrieve_from_web(q: str, top_k: int = 2) -> List[Tuple[str, float]]:
    """Web search (limité pour éviter rate limit)"""
    results = []
    try:
        search_results = list(search(q, num_results=top_k, lang="en"))

        for url in search_results[:top_k]:
            try:
                response = requests.get(url, timeout=3)
                soup = BeautifulSoup(response.text, 'html.parser')
                paragraphs = soup.find_all('p')
                text = ' '.join([p.get_text() for p in paragraphs[:2]])
                if text:
                    results.append((text[:300], 0.65))
            except:
                continue
    except Exception as e:
        print(f"⚠️ Web search failed: {e}")

    return results


def rerank_documents(q: str, docs: List[Tuple[str, float]], top_k: int = 3) -> List[Tuple[str, float]]:
    """Reranking sémantique"""
    if not docs:
        return []

    texts = [d for d, _ in docs]
    qv = model.encode(q, normalize_embeddings=True)
    dv = model.encode(texts, normalize_embeddings=True)
    scores = np.dot(dv, qv)

    ranked = sorted(zip(texts, scores), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]


def evaluate_confidence(ranked: List[Tuple[str, float]]) -> float:
    """Évalue la confiance des résultats"""
    if not ranked:
        return 0.0

    # Score basé sur le meilleur match
    best_score = ranked[0][1]

    # Bonus si plusieurs docs cohérents
    if len(ranked) >= 2 and ranked[1][1] > 0.6:
        best_score += 0.1

    return min(best_score, 0.95)


# === ORCHESTRATEUR PRINCIPAL ===

def orchestrator(question: str) -> Dict:
    """
    Orchestrateur intelligent qui décide dynamiquement de la stratégie
    """
    print(f"\n{'='*70}")
    print(f"🎯 ORCHESTRATEUR ACTIVÉ")
    print(f"{'='*70}")

    # 1. ANALYSE
    print(f"\n📋 ÉTAPE 1 : Analyse de la question")
    action = analyze_question(question)
    print(f"   → Domaine: {action.domaine}")
    print(f"   → Module initial: {action.selection_module.upper()}")
    print(f"   → Confiance: {action.confidence:.2f}")

    all_docs = []
    strategies_used = []

    # 2. STRATÉGIE PRIMAIRE
    print(f"\n🔍 ÉTAPE 2 : Récupération ({action.selection_module})")

    if action.selection_module == "text":
        docs_text = retrieve_from_text(question)
        all_docs.extend(docs_text)
        strategies_used.append("text_search")
        print(f"   → {len(docs_text)} docs de Qdrant")

    elif action.selection_module == "graph":
        docs_graph = retrieve_from_graph(question)
        all_docs.extend(docs_graph)
        strategies_used.append("graph_search")
        print(f"   → {len(docs_graph)} relations du graphe")

    elif action.selection_module == "web":
        docs_web = retrieve_from_web(question)
        all_docs.extend(docs_web)
        strategies_used.append("web_search")
        print(f"   → {len(docs_web)} résultats web")

    # 3. RERANKING
    print(f"\n📊 ÉTAPE 3 : Reranking")
    ranked = rerank_documents(question, all_docs)
    confidence = evaluate_confidence(ranked)
    print(f"   → {len(ranked)} docs après reranking")
    print(f"   → Confiance: {confidence:.2%}")

    # 4. STRATÉGIE SECONDAIRE (si confiance faible)
    if confidence < 0.65 and action.selection_module != "graph":
        print(f"\n🔄 ÉTAPE 4 : Confiance faible → Essai GRAPHE")
        docs_graph = retrieve_from_graph(question)
        all_docs.extend(docs_graph)
        strategies_used.append("graph_fallback")

        ranked = rerank_documents(question, all_docs, top_k=5)
        confidence = evaluate_confidence(ranked)
        print(f"   → Nouvelle confiance: {confidence:.2%}")

    # 5. STRATÉGIE TERTIAIRE (si toujours faible)
    if confidence < 0.5 and "web_search" not in strategies_used:
        print(f"\n🌐 ÉTAPE 5 : Confiance critique → WEB SEARCH")
        docs_web = retrieve_from_web(question)
        all_docs.extend(docs_web)
        strategies_used.append("web_emergency")

        ranked = rerank_documents(question, all_docs, top_k=5)
        confidence = evaluate_confidence(ranked)
        print(f"   → Confiance finale: {confidence:.2%}")

    # 6. GÉNÉRATION
    print(f"\n🤖 ÉTAPE 6 : Génération de la réponse")
    if ranked:
        context = "\n".join([f"[{i+1}] {doc[:200]}" for i, (doc, _) in enumerate(ranked[:3])])
        prompt = f"""Question: {question}

Context from multiple sources:
{context}

Provide a clear, concise answer based on the context:"""

        answer = generate_with_llm(prompt, max_tokens=120)
    else:
        answer = "❌ Aucune information pertinente trouvée."

    # 7. RÉSULTAT
    result = {
        "question": question,
        "answer": answer,
        "confidence": confidence,
        "strategies_used": strategies_used,
        "sources": ranked,
        "domain": action.domaine
    }

    return result


# === FONCTION D'AFFICHAGE ===

def ask(question: str):
    """Interface utilisateur pour l'orchestrateur"""
    if not question.strip():
        print("⚠️ Pose une question valide !")
        return

    result = orchestrator(question)

    print(f"\n{'='*70}")
    print(f"✅ RÉPONSE FINALE")
    print(f"{'='*70}")
    print(f"\n{result['answer']}")
    print(f"\n📈 Confiance: {result['confidence']:.2%}")
    print(f"🔧 Stratégies utilisées: {', '.join(result['strategies_used'])}")
    print(f"📚 Sources: {len(result['sources'])} documents")

    print(f"\n📄 Top 2 sources:")
    for i, (doc, score) in enumerate(result['sources'][:2], 1):
        print(f"   [{i}] Score: {score:.3f}")
        print(f"       {doc[:150]}...")
    print(f"{'='*70}\n")


print("✅ ORCHESTRATEUR PRÊT")

✅ ORCHESTRATEUR PRÊT


In [ ]:
# CELLULE 8 : Interface Interactive
# ========================================
from IPython.display import display
import ipywidgets as widgets

txt = widgets.Text(
    placeholder="Ex: How do nanofluids improve heat transfer?",
    description="Question:",
    style={'description_width': '80px'},
    layout=widgets.Layout(width='700px')
)

btn = widgets.Button(
    description="🚀 Ask CRAG",
    button_style="success",
    layout=widgets.Layout(width='150px')
)

out = widgets.Output()

def on_click(b):
    with out:
        out.clear_output()
        if txt.value.strip():
            ask(txt.value)
        else:
            print("⚠️ Entre une question d'abord !")

btn.on_click(on_click)

print("✅ Interface prête:")
display(widgets.VBox([txt, btn, out]))

✅ Interface prête:


In [ ]:
# CELLULE 9 : Tests de l'Orchestrateur
# ========================================
test_questions = [
    # Test 1: Question simple (text search)
    "What are nanofluids?",

    # Test 2: Question relationnelle (graph)
    "What is the relation between copper oxide and thermal conductivity?",

    # Test 3: Question complexe (multi-strategy)
    "How do nanoparticles enhance heat transfer efficiency in thermal systems?"
]

print("🧪 TESTS DE L'ORCHESTRATEUR\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'#'*70}")
    print(f"TEST {i}/3")
    print(f"{'#'*70}")
    ask(q)
    print("\n" + "="*70 + "\n")

🧪 TESTS DE L'ORCHESTRATEUR


######################################################################
TEST 1/3
######################################################################

🎯 ORCHESTRATEUR ACTIVÉ

📋 ÉTAPE 1 : Analyse de la question
   → Domaine: Thermal
   → Module initial: TEXT
   → Confiance: 0.80

🔍 ÉTAPE 2 : Récupération (text)


AttributeError: 'QdrantClient' object has no attribute 'search'